### Extracting openmm system force information and adding forces

scripts adapted from https://github.com/choderalab/alchemical-playground/blob/master/protein-mutations/annihilation-test.py

In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

from openmmtools.testsystems import *

# importing restraints script
from restraints import *

import math

In [2]:
alanine = AlanineDipeptideVacuum()
ala_system, ala_positions, ala_topology = alanine.system, alanine.positions,  alanine.topology

In [3]:
for force in ala_system.getForces():
    print(force.__class__.__name__)

HarmonicBondForce
HarmonicAngleForce
PeriodicTorsionForce
NonbondedForce
CMMotionRemover


In [4]:
atom_dict = {}
for atom in ala_topology.atoms():
    atom_dict[atom.index] = [atom.name,atom.residue.name]

In [5]:
# bond forces 
for force in ala_system.getForces():
    if isinstance(force, HarmonicBondForce):
        print('%10s%10s%13s' % ('Atoms', 'bonds', 'Force'))
        for bond_index in range(force.getNumBonds()):
            [iatom, jatom, distance, k] = force.getBondParameters(bond_index)
            print('%5d%5d%10.3f A%10.3f kcal/mol/A**2' % (iatom, jatom, distance.value_in_unit(angstrom), k.value_in_unit(kilocalories_per_mole/angstroms**2)))
            print('%5s%5s' % (atom_dict[iatom][0], atom_dict[jatom][0]))
            print('%5s%5s\n' % (atom_dict[iatom][1], atom_dict[jatom][1]))
            

     Atoms     bonds        Force
    4    5     1.229 A  1140.000 kcal/mol/A**2
    C    O
  ACE  ACE

    4    6     1.335 A   980.000 kcal/mol/A**2
    C    N
  ACE  ALA

    1    4     1.522 A   634.000 kcal/mol/A**2
  CH3    C
  ACE  ACE

   14   15     1.229 A  1140.000 kcal/mol/A**2
    C    O
  ALA  ALA

   14   16     1.335 A   980.000 kcal/mol/A**2
    C    N
  ALA  NME

    8   10     1.526 A   620.000 kcal/mol/A**2
   CA   CB
  ALA  ALA

    8   14     1.522 A   634.000 kcal/mol/A**2
   CA    C
  ALA  ALA

    6    8     1.449 A   674.000 kcal/mol/A**2
    N   CA
  ALA  ALA

   16   18     1.449 A   674.000 kcal/mol/A**2
    N    C
  NME  NME



In [6]:
# angle forces 
for force in ala_system.getForces():
    if isinstance(force, HarmonicAngleForce):
        print('%15s%10s%15s' % ('Atoms', 'Angle', 'Force'))
        for angle_index in range(force.getNumAngles()):
            [iatom, jatom, katom, theta0, Ktheta] = force.getAngleParameters(angle_index)
            print('%5d%5d%5d%10.3f rad %10.3f kcal/mol/rad**2' % (iatom, jatom, katom, theta0.value_in_unit(radian), Ktheta.value_in_unit(kilocalories_per_mole/radian**2)))
            print('%5s%5s%5s' % (atom_dict[iatom][0], atom_dict[jatom][0], atom_dict[katom][0]))
            print('%5s%5s%5s\n' % (atom_dict[iatom][1], atom_dict[jatom][1], atom_dict[katom][1]))

          Atoms     Angle          Force
    4    6    7     2.094 rad     60.000 kcal/mol/rad**2
    C    N    H
  ACE  ALA  ALA

    3    1    4     1.911 rad    100.000 kcal/mol/rad**2
   H3  CH3    C
  ACE  ACE  ACE

    2    1    3     1.911 rad     70.000 kcal/mol/rad**2
   H2  CH3   H3
  ACE  ACE  ACE

    2    1    4     1.911 rad    100.000 kcal/mol/rad**2
   H2  CH3    C
  ACE  ACE  ACE

    0    1    2     1.911 rad     70.000 kcal/mol/rad**2
   H1  CH3   H2
  ACE  ACE  ACE

    0    1    3     1.911 rad     70.000 kcal/mol/rad**2
   H1  CH3   H3
  ACE  ACE  ACE

    0    1    4     1.911 rad    100.000 kcal/mol/rad**2
   H1  CH3    C
  ACE  ACE  ACE

   14   16   17     2.094 rad     60.000 kcal/mol/rad**2
    C    N    H
  ALA  NME  NME

   12   10   13     1.911 rad     70.000 kcal/mol/rad**2
  HB2   CB  HB3
  ALA  ALA  ALA

   11   10   12     1.911 rad     70.000 kcal/mol/rad**2
  HB1   CB  HB2
  ALA  ALA  ALA

   11   10   13     1.911 rad     70.000 kcal/mol/rad**2
  

In [7]:
for force in ala_system.getForces():
    if isinstance(force, PeriodicTorsionForce):
        print('%20s%15s%10s%15s' % ('Atoms', 'periodicity', 'Angle', 'Force'))
        for torsion_index in range(force.getNumTorsions()):
            particle1, particle2, particle3, particle4, periodicity, phase, k = force.getTorsionParameters(torsion_index)
            print('%5d%5d%5d%5d%15s%10.3f rad %10.3f kcal/mol' % (particle1, particle2, particle3, particle4, periodicity, phase.value_in_unit(radian), k.value_in_unit(kilocalories_per_mole)))
            print('%5s%5s%5s%5s' % (atom_dict[particle1][0], atom_dict[particle2][0], atom_dict[particle3][0], atom_dict[particle4][0]))
            print('%5s%5s%5s%5s\n' % (atom_dict[particle1][1], atom_dict[particle2][1], atom_dict[particle3][1], atom_dict[particle4][1]))

               Atoms    periodicity     Angle          Force
    5    4    6    7              1     0.000 rad      2.000 kcal/mol
    O    C    N    H
  ACE  ACE  ALA  ALA

    5    4    6    7              2     3.142 rad      2.500 kcal/mol
    O    C    N    H
  ACE  ACE  ALA  ALA

    4    6    8    9              2     0.000 rad      0.000 kcal/mol
    C    N   CA   HA
  ACE  ALA  ALA  ALA

    3    1    4    5              2     0.000 rad      0.000 kcal/mol
   H3  CH3    C    O
  ACE  ACE  ACE  ACE

    3    1    4    6              2     0.000 rad      0.000 kcal/mol
   H3  CH3    C    N
  ACE  ACE  ACE  ALA

    2    1    4    5              2     0.000 rad      0.000 kcal/mol
   H2  CH3    C    O
  ACE  ACE  ACE  ACE

    2    1    4    6              2     0.000 rad      0.000 kcal/mol
   H2  CH3    C    N
  ACE  ACE  ACE  ALA

    1    4    6    7              2     3.142 rad      2.500 kcal/mol
  CH3    C    N    H
  ACE  ACE  ALA  ALA

    0    1    4    5              2

In [8]:
for force in ala_system.getForces():
    if isinstance(force, NonbondedForce):
        print('%5s%10s%14s%19s' % ('Atom','Charge', 'vdw-radius', 'vdw-well_depth'))
        for particle_index in range(force.getNumParticles()):
                # Retrieve parameters.
                [charge, sigma, epsilon] = force.getParticleParameters(particle_index)
                print('%5d%10.3f e%12.3f nm%16.3f kJ/mol' % (particle_index,charge.value_in_unit(elementary_charge),sigma.value_in_unit(angstrom),epsilon.value_in_unit(kilojoule_per_mole)))
                

 Atom    Charge    vdw-radius     vdw-well_depth
    0     0.112 e       2.650 nm           0.066 kJ/mol
    1    -0.366 e       3.400 nm           0.458 kJ/mol
    2     0.112 e       2.650 nm           0.066 kJ/mol
    3     0.112 e       2.650 nm           0.066 kJ/mol
    4     0.597 e       3.400 nm           0.360 kJ/mol
    5    -0.568 e       2.960 nm           0.879 kJ/mol
    6    -0.416 e       3.250 nm           0.711 kJ/mol
    7     0.272 e       1.069 nm           0.066 kJ/mol
    8     0.034 e       3.400 nm           0.458 kJ/mol
    9     0.082 e       2.471 nm           0.066 kJ/mol
   10    -0.182 e       3.400 nm           0.458 kJ/mol
   11     0.060 e       2.650 nm           0.066 kJ/mol
   12     0.060 e       2.650 nm           0.066 kJ/mol
   13     0.060 e       2.650 nm           0.066 kJ/mol
   14     0.597 e       3.400 nm           0.360 kJ/mol
   15    -0.568 e       2.960 nm           0.879 kJ/mol
   16    -0.416 e       3.250 nm           0.711 kJ/mol

In [9]:
for force in ala_system.getForces():
    if isinstance(force, NonbondedForce):
        print('%10s%15s%14s%19s' % ('Atom','Charge-prod', 'vdw-radius', 'vdw-well_depth'))
        for exception_index in range(force.getNumExceptions()):
            [iatom, jatom, chargeprod, sigma, epsilon] = force.getExceptionParameters(exception_index)
            print('%5d%5d%15.3f e%12.3f nm%16.3f kJ/mol' % (iatom,jatom,charge.value_in_unit(elementary_charge),sigma.value_in_unit(angstrom),epsilon.value_in_unit(kilojoule_per_mole)))

      Atom    Charge-prod    vdw-radius     vdw-well_depth
    5    7          0.098 e       2.015 nm           0.120 kJ/mol
    4    9          0.098 e       2.936 nm           0.077 kJ/mol
    3    5          0.098 e       2.805 nm           0.120 kJ/mol
    3    6          0.098 e       2.950 nm           0.108 kJ/mol
    2    5          0.098 e       2.805 nm           0.120 kJ/mol
    2    6          0.098 e       2.950 nm           0.108 kJ/mol
    1    7          0.098 e       2.234 nm           0.087 kJ/mol
    0    5          0.098 e       2.805 nm           0.120 kJ/mol
    0    6          0.098 e       2.950 nm           0.108 kJ/mol
   15   17          0.098 e       2.015 nm           0.120 kJ/mol
   14   19          0.098 e       2.936 nm           0.077 kJ/mol
   14   20          0.098 e       2.936 nm           0.077 kJ/mol
   14   21          0.098 e       2.936 nm           0.077 kJ/mol
   13   14          0.098 e       3.025 nm           0.077 kJ/mol
   12   14       

In [10]:
# Adding custom forces
# We are adding forces that already exist in the system, this is a simple demonstration
# The functions require a PDB file to access both topology and positions
# A more elaborate application is found the OpenMM_alchemical_scripts repository
PDBFile.writeFile(ala_topology,ala_positions,open('ala.pdb', 'w+'))

In [11]:
pdb = PDBFile('ala.pdb')

In [12]:
ace_atoms = [atom.index for atom in ala_topology.atoms() if atom.residue.name == 'ACE']
alanine_atoms = [atom.index for atom in ala_topology.atoms() if atom.residue.name == 'ALA']
nme_atoms = [atom.index for atom in ala_topology.atoms() if atom.residue.name == 'NME']

In [13]:
# positional restraint on ACE atoms
ala_system = pos_rest(ala_system, pdb, ace_atoms, force=1, force_var='ala_pos_force')
# Z_direction restraint on nme atoms
ala_system = z_rest(ala_system, pdb, alanine_atoms, force=2)
# Flat-bottom group centroid restraints on ALA and NME
#ala_system = fb_groups_rest(ala_system,pdb,[alanine_atoms,nme_atoms], dist_var='fb_group_distance', force_var='fb_group_force', distance=3, force=1)
# Flat-bottom atom restrains
ala_system = fb_atoms_rest(ala_system,pdb,[4,5],dist_var='fb_atom_distance', force_var='fb_atom_force', distance=1.229, force=1140)
# Angle force, 2 given
ala_system = angle_rest(ala_system,pdb,[[4,6,7],[3,1,4]],angle_var='eq_angle', force_var='angle_force', angle=2.094, force=60)
# Torsion force
ala_system = torsion_rest(ala_system,pdb,[5,4,6,7],angle_var='eq_torsion', force_var='torsion_force', angle=math.pi, force=2.5)


Applying positional restraint force:
force_variable: ala_pos_force = 1*kilocalories_per_mole/angstroms**2
Applying Z-direction restraint force:
force_variable: z_force = 2*kilocalories_per_mole/angstroms**2
Applying Flat-bottom atom restraint force:
force_variable: fb_atom_force = 1140*kilocalories_per_mole/angstroms**2
distance_variable: fb_atom_distance = 1*Angstrom
Applying Angle restraint force:
force_variable: angle_force = 60*kilocalories_per_mole/radian**2
angle_variable: eq_angle = 2*radians
Applying Torsional restraint force:
force_variable: torsion_force = 2*kilocalories_per_mole
angle_variable: eq_torsion = 3*radians


In [14]:
system_forces = forces()

In [15]:
system_forces

{'ala_pos_force': '1*kilocalories_per_mole/angstroms**2',
 'z_force': '2*kilocalories_per_mole/angstroms**2',
 'fb_atom_force': '1140*kilocalories_per_mole/angstroms**2',
 'fb_atom_distance': '1.229*Angstrom',
 'angle_force': '60*kilocalories_per_mole/radian**2',
 'eq_angle': '2.094*radians',
 'torsion_force': '2.5*kilocalories_per_mole',
 'eq_torsion': '3.141592653589793*radians'}

In [16]:
for force in ala_system.getForces():
    print(force.__class__.__name__)

HarmonicBondForce
HarmonicAngleForce
PeriodicTorsionForce
NonbondedForce
CMMotionRemover
CustomExternalForce
CustomExternalForce
CustomBondForce
CustomAngleForce
CustomTorsionForce


In [17]:
integrator = LangevinIntegrator(300*kelvin,5/picosecond,1*femtosecond)

In [18]:
simulation = Simulation(ala_topology,ala_system,integrator)

In [19]:
simulation.context.setPositions(pdb.positions)

In [20]:
simulation.minimizeEnergy()

In [21]:
simulation.reporters.append(DCDReporter('ala_sim.dcd',100))
simulation.reporters.append(StateDataReporter('ala_sim.log', 100,
                                              step=True,
                                              time=True,
                                              speed=True,
                                              potentialEnergy=True,
                                              kineticEnergy=True,
                                              temperature=True,
                                              volume=True,
                                              progress=True,
                                              remainingTime=True,
                                              totalSteps=10000,
                                              separator='\t'))

In [22]:
simulation.step(10000)